
# 第2课：贝尔曼公式（Bellman Equation）

贝尔曼公式是强化学习中最核心的数学工具，它通过**递归定义值函数**，为策略评估与策略改进提供了基础。

---

## Part 1：例子说明 Return 的重要性

在强化学习中，智能体的目标不是最大化**即时奖励**，而是最大化从当前开始的**累积回报（return）**。

### 示例：走迷宫

假设智能体需要从起点走到终点，每一步的奖励是 -1，终点的奖励是 +10。以下是两条路径：

* 路径 A：在 5 步内走到终点，Return = $-1 \times 4 + 10 = 6$
* 路径 B：绕远路，10 步到终点，Return = $-1 \times 9 + 10 = 1$

虽然终点奖励相同，但路径 A 明显更优，因为其 **Return 更大**。

这表明：

> **回报 $G_t$** 才是衡量策略优劣的真正标准，而不是单步奖励。

### deterministic or stochastic？

对确定的初始状态和轨迹，return可以用来评估策略。

对随机的初始状态和轨迹，如何评估策略？————state value



## Part 2：State Value or State Value Function 的定义（状态值函数）

---

值函数衡量一个状态“好不好”，即从该状态开始，**跟随策略 $\pi$** 所能获得的**期望回报**。

换句话说，值函数不是单步奖励的期望，而是**未来所有奖励的加权期望和**，是智能体在当前状态下、执行某一策略时未来行为的整体“质量评估”。

---

### 定义：

$$
\boxed{
v_{\pi}(s) = \mathbb{E}_\pi \left[ G_t \mid s_t = s \right]
= \mathbb{E}_\pi \left[ \sum_{k=0}^\infty \gamma^k r_{t+k+1} \,\Big|\, s_t = s \right]
}
$$

其中：

* $V_{\pi}(s)$：状态 $s$ 在策略 $\pi$ 下的值
* $\gamma \in [0,1]$：折扣因子
* $G_t$：从时刻 $t$ 开始的**折扣回报**
* 期望是对策略 $\pi$ 引导下的所有可能轨迹求的平均



> 如何求解值函数？——贝尔曼方程 & Bootstrapping


### Bootstrapping 的概念

> **Bootstrapping** 指在估计当前状态值时，**用到了对下一个状态值的估计**，而不是等待完整回报（episode结束）来计算。

这是**动态规划、时序差分（TD）、Q-learning 等算法的核心特征**：

* 与蒙特卡洛方法不同，bootstrapping 不需要等待 episode 完结即可更新值
* 提高了样本效率，也使得算法能进行**在线学习**

---

### 举例：TD(0) 使用 bootstrapping 更新 $v_{\pi}(s)$

$$
v(s_t) \leftarrow v(s_t) + \alpha \big[ r_{t+1} + \gamma v(s_{t+1}) - v(s_t) \big]
$$

* 当前值的更新用到了**对下一个状态值的当前估计**
* 所以这个更新使用了 bootstrapping

---

## 小结：

* 状态值函数 $v_{\pi}(s)$ 表示从状态 $s$ 出发，在策略 $\pi$ 下的期望累计回报
* 贝尔曼方程揭示了它的递归结构，是求解的核心依据
* **Bootstrapping** 是利用已有估计值来加速学习、避免等待完整回报的重要方法，是多数强化学习算法的基础


## Part 3：贝尔曼公式的详细推导（对状态值）

In a nutshell, 贝尔曼方程是一组描述所有状态值之间关系的线性方程。

贝尔曼公式利用了**值函数的递归结构**：当前状态的值 = 当前期望奖励 + 未来状态的值（加折扣）。

$$
\boxed{
v_{\pi}(s) = \mathbb{E}_{a \sim \pi(\cdot|s)} \left[ r(s,a) + \gamma \mathbb{E}_{s' \sim p(\cdot|s,a)} [v_{\pi}(s')] \right]
}
$$

也可展开写为：

$$
v_{\pi}(s) = \sum_{a \in \mathcal{A}} \pi(a|s) \left[ r(s,a) + \gamma \sum_{s'} p(s'|s,a) v_{\pi}(s') \right]
$$




### 推导直觉(计算贝尔曼公式所需要的三个概率值)：

1. 在状态 $s$ 下，策略选择动作 $a$，概率为 $\pi(a|s)$
2. 环境反馈奖励 $r(s,a)$，并转移到新状态 $s'$，概率为 $p(s'|s,a)$
3. 在新状态 $s'$ 上继续累计回报 $v^\pi(s')$，但要乘以折扣 $\gamma$


### Bellman equation详细推导

![](./img/2_1.png)
![](./img/2_2.png)
![](./img/2_3.png)
![](./img/2_4.png)
![](./img/2_5.png)

\begin{aligned}
v_{\pi}(s) & =\mathbb{E}[R_{t+1}|S_{t}=s]+\gamma\mathbb{E}[G_{t+1}|S_{t}=s], \\
 & =\underbrace{\sum_{a\in\mathcal{A}}\pi(a|s)\sum_{r\in\mathcal{R}}p(r|s,a)r}_{\text{mean of immediate rewards}}+\underbrace{\gamma\sum_{a\in\mathcal{A}}\pi(a|s)\sum_{s^{\prime}\in\mathcal{S}}p(s^{\prime}|s,a)v_{\pi}(s^{\prime})}_{\text{mean of future rewards}} \\
 & =\sum_{a\in\mathcal{A}}\pi(a|s)\left[\sum_{r\in\mathcal{R}}p(r|s,a)r+\gamma\sum_{s^{\prime}\in\mathcal{S}}p(s^{\prime}|s,a)v_{\pi}(s^{\prime})\right],\quad\text{for all }s\in\mathcal{S}.
\end{aligned}




## Part 4：向量形式与线性方程求解

当状态空间 $\mathcal{S}$ 有限时，可将值函数表示为向量 $\mathbf{v}$，贝尔曼公式可以写成矩阵形式：

### 向量化表示：

$$
\mathbf{v}_{\pi} = \mathbf{r}_{\pi} + \gamma \mathbf{P}_{\pi} \mathbf{v}_{\pi}
$$


* $\mathbf{v}_{\pi}$：状态值向量，维度为 $|\mathcal{S}|$，（未知）
* $\mathbf{r}^\pi$：每个状态下，按策略 $\pi$ 的期望奖励，（已知）
* $\mathbf{P}^\pi$：状态转移矩阵，维度为 $|\mathcal{S}| \times |\mathcal{S}|$，（已知）

### 线性代数解法：

将上式移项：

$$
(I - \gamma \mathbf{P}_{\pi})\mathbf{v}_{\pi} = \mathbf{r}_{\pi}
\quad \Rightarrow \quad
\boxed{
\mathbf{v}_{\pi} = (I - \gamma \mathbf{P}_{\pi})^{-1} \mathbf{r}_{\pi}
}
$$

![](./img/2_6.png)

因为该解法包含矩阵求逆的运算，因此仅适用于小规模状态空间，在大规模（例如图像状态）中不可行。

### 迭代解法

![](./img/2_7.png)


## Part 5：Action Value 的定义（动作值函数）

状态值函数 $V_{\pi}(s)$ 描述“在状态 $s$ 开始的好坏”；而**动作值函数** $Q_{\pi}(s,a)$ 描述“在状态 $s$，执行动作 $a$ 后的好坏”。

### 定义：

$$
Q_{\pi}(s,a) = \mathbb{E}_\pi \left[ G_t \mid s_t = s, a_t = a \right]
= \mathbb{E} \left[ r_{t+1} + \gamma V_{\pi}(s_{t+1}) \,\big|\, s_t = s, a_t = a \right]
$$

或递归式写为：

$$
Q_{\pi}(s,a) = R(s,a) + \gamma \sum_{s'} P(s'|s,a) V_{\pi}(s')
$$

### 状态值与动作值的关系：

$$
V_{\pi}(s) = \sum_a \pi(a|s) Q_{\pi}(s,a)
$$

这意味着：状态值是**在当前策略下，对动作值函数的加权平均**。



## Part 6：The Bellman Equation in Terms of Action Values

（动作值函数形式下的贝尔曼方程）

### 回顾：

动作值函数 $Q_\pi(s,a)$ 表示：在状态 $s$ 下采取动作 $a$，然后在策略 $\pi$ 下继续行动所能获得的**期望回报**。
我们希望写出类似于状态值函数的贝尔曼方程，但用 $Q_\pi(s,a)$ 表示。

---

### 动作值形式的贝尔曼方程：

$$
Q_\pi(s,a) = \mathbb{E}_\pi \left[ r_{t+1} + \gamma Q_\pi(s_{t+1}, a_{t+1}) \,\big|\, s_t = s, a_t = a \right]
$$

这是对 $Q_\pi(s,a)$ 的递归定义，和状态值的贝尔曼方程类似，但现在考虑的是动作对未来价值的影响。

---

### 展开写法（使用转移概率和策略）：

$$
Q_\pi(s,a) = R(s,a) + \gamma \sum_{s'} P(s' \mid s,a) \sum_{a'} \pi(a' \mid s') Q_\pi(s', a')
$$

其中：

* $R(s,a) = \mathbb{E}[r_{t+1} \mid s_t=s, a_t=a]$：状态-动作对的期望奖励
* $P(s' \mid s,a)$：执行 $a$ 后到达下一个状态 $s'$ 的概率
* $\pi(a' \mid s')$：策略在 $s'$ 状态下选择 $a'$ 的概率
* $Q_\pi(s', a')$：在下一个状态-动作对的动作值函数

---

### 理解：值函数的递归结构（Bootstrapping）

* 当前 $Q_\pi(s,a)$ 的估计依赖于下一时刻 $Q_\pi(s',a')$ 的值
* 这是一种 **Bootstrapping（自举）**：用自己的近似值来更新自身
* 是强化学习中迭代计算策略价值的核心思想

---

### 与状态值形式的贝尔曼方程的比较

| 比较维度            | 状态值形式（State Value）                                          | 动作值形式（Action Value）                                                                 |
|---------------------|---------------------------------------------------------------------|---------------------------------------------------------------------------------------------|
| **定义目标**        | 从状态 $s$ 出发，在策略 $\pi$ 下获得的期望回报                     | 从状态 $s$ 采取动作 $a$ 后，在策略 $\pi$ 下获得的期望回报                                    |
| **递归关系**        | $V_\pi(s) = \sum_a \pi(a|s) Q_\pi(s,a)$<br>或<br>$V_\pi(s) = R(s) + \gamma \sum_{s'} P(s'|s) V_\pi(s')$ | $Q_\pi(s,a) = R(s,a) + \gamma \sum_{s'} P(s'|s,a) \sum_{a'} \pi(a'|s') Q_\pi(s',a')$         |
| **价值更新**        | 值依赖于当前状态的后继状态                                          | 值依赖于当前状态和当前动作的后继状态与动作                                                  |
| **常用于**          | 评估策略，或用于策略迭代中的 $V \rightarrow \pi$ 更新               | 用于策略评估和策略改进，尤其是基于动作值的学习算法（如 Sarsa、Q-learning）                  |
| **是否涉及策略概率**| 显式使用策略 $\pi(a|s)$ 进行动作加权                                 | 显式使用策略 $\pi(a'|s')$ 在后续状态中加权所有动作                                          |
| **Bootstrapping**   | $V_\pi(s)$ 由 $V_\pi(s')$ 递归计算得出                               | $Q_\pi(s,a)$ 由 $Q_\pi(s',a')$ 递归计算得出                                                 |


---

### 小结：

* 动作值的贝尔曼方程提供了对 $Q_\pi(s,a)$ 的递归定义
* 它是实现许多强化学习算法（如 **Sarsa**、**Q-learning**）的理论基础
* 该方程支持通过经验或采样来估计值函数并改进策略



### 小结

| 函数                   | 解释                         | 数学定义                                                                                                    |
|----------------------|------------------------------|---------------------------------------------------------------------------------------------------------|
| 状态值函数 $V_{\pi}(s)$   | 在状态 $s$ 下，期望的长期奖励      | $V_{\pi}(s) = \mathbb{E}_\pi \left[ G_t \mid s_t = s \right]$                                           |
| 动作值函数 $Q_{\pi}(s,a)$ | 在状态 $s$ 下采取动作 $a$ 后的长期奖励 | $Q_{\pi}(s,a) = \mathbb{E}_\pi \left[ G_t \mid s_t = s, a_t = a \right]$                                |
| 贝尔曼方程                | 定义值函数的递归关系             | $V_{\pi}(s) = \sum_a \pi(a \mid s) \left[ R(s,a) + \gamma \sum_{s'} P(s' \mid s,a) V_{\pi}(s') \right]$ |


# Q1 状态值（state value）, 贝尔曼方程（bellman equation）, 策略评估（policy evaluation）的关系？

这是强化学习中一个非常核心的问题，下面我来**系统地梳理**一下 **state value（状态值函数）**、**Bellman Equation（贝尔曼方程）** 和 **Policy Evaluation（策略评估）** 三者之间的关系：

---

## 一、state value：状态值函数 $v_{\pi}(s)$

### 定义：

在给定策略 $\pi$ 下，从状态 $s$ 开始，智能体将获得的**期望回报（expected return）**：

$$
\boxed{
V^\pi(s) = \mathbb{E}_\pi \left[ \sum_{t=0}^{\infty} \gamma^t r_{t+1} \mid s_0 = s \right]
}
$$

* 它评估了：**如果我当前处于状态 $s$，然后按照策略 $\pi$ 行动，我预计能获得多大回报？**

---

## 二、Bellman Equation：贝尔曼方程

### 定义：

贝尔曼方程是对值函数的**递归定义**，把 $V^\pi(s)$ 表达为当前奖励和下一个状态的值的组合：

$$
\boxed{
V^\pi(s) = \sum_{a} \pi(a|s) \sum_{s'} P(s'|s,a) \left[ R(s,a,s') + \gamma V^\pi(s') \right]
}
$$

其中：

* $\pi(a|s)$：当前策略
* $P(s'|s,a)$：环境动态（状态转移概率）
* $R(s,a,s')$：从 $s$ 采取 $a$ 到 $s'$ 的即时奖励
* $\gamma$：折扣因子

---

### 解读：

贝尔曼方程给出了 **“状态值函数的自洽条件”**，它说明了状态值应该等于：

> “在该状态下，期望采取某个动作、获得奖励，加上到达下一状态后继续执行该策略所获得的价值”。

---

## 三、Policy Evaluation：策略评估

### 定义：

**在已知策略 $\pi$** 的情况下，求出所有状态的值函数 $V^\pi(s)$，这一步叫做策略评估。

---

### 方法：

通常通过迭代方法来求解贝尔曼方程：

$$
V_{k+1}(s) = \sum_{a} \pi(a|s) \sum_{s'} P(s'|s,a) \left[ R(s,a,s') + \gamma V_k(s') \right]
$$

直到：

$$
V_{k+1}(s) \approx V_k(s)
$$

这就是**迭代式策略评估（Iterative Policy Evaluation）**，也是策略迭代和价值迭代算法中的一个基本步骤。

---

## 三者关系总结

| 概念                   | 作用                  | 相互关系              |
| -------------------- | ------------------- | ----------------- |
| **状态值函数** $V^\pi(s)$ | 衡量某状态的好坏（期望回报）      | 是目标函数             |
| **贝尔曼方程**            | 给出 $V^\pi(s)$ 的递推定义 | 是对 $V^\pi(s)$ 的约束 |
| **策略评估**             | 计算出 $V^\pi(s)$ 的数值解 | 是求解贝尔曼方程的过程       |

---

### 类比图示理解：

```text
   策略 π
     ↓
状态值函数 Vπ(s) ←——— 贝尔曼方程（定义 Vπ 的递归结构）
     ↑
 策略评估（迭代或解线性系统求解 Vπ）
```




# Q2 贝尔曼既可以用状态值表示，也可以用动作值表示？

贝尔曼方程（Bellman Equation）确实可以根据我们选择的值函数表示方式，**既可以用状态值（State Value）表示，也可以用动作值（Action Value）表示**，它们本质上都表达了“当前值 = 当前奖励 + 折扣后的未来值”这一核心思想。

---

## 一句话总结：

> **贝尔曼方程是一种对“期望回报”的递归定义，可以基于状态值函数 $V^\pi(s)$ 或动作值函数 $Q^\pi(s,a)$ 来表示。**

---

## 两种形式对比说明：

| 形式           | 名称               | 数学定义                                                                                 | 意义 |
|----------------|--------------------|------------------------------------------------------------------------------------------|------|
| 状态值形式     | 贝尔曼状态值方程   | $V^\pi(s) = \sum_a \pi(a \mid s) \left[ R(s,a) + \gamma \sum_{s'} P(s' \mid s,a) V^\pi(s') \right]$ | 表示在状态 $s$ 下，跟随策略 $\pi$ 所期望获得的总回报 |
| 动作值形式     | 贝尔曼动作值方程   | $Q^\pi(s,a) = R(s,a) + \gamma \sum_{s'} P(s' \mid s,a) \sum_{a'} \pi(a' \mid s') Q^\pi(s',a')$     | 表示在状态 $s$ 下执行动作 $a$，并随后按策略 $\pi$ 行动所期望的总回报 |


---

## 关系：

* 状态值可以由动作值得到：

  $$
  V^\pi(s) = \sum_a \pi(a|s) Q^\pi(s,a)
  $$

* 状态值方程可以看作是“在动作值空间中对策略概率的加权平均”。

---

## 为什么要有两种形式？

| 角度             | 状态值形式                                 | 动作值形式                                                  |
|------------------|--------------------------------------------|-------------------------------------------------------------|
| **可用性**        | 若策略已知且不需直接优化动作，状态值足够              | 若要优化策略或找最优动作，必须使用动作值                            |
| **典型算法**      | 策略迭代、值迭代                              | Sarsa、Q-learning、Actor-Critic 等                          |
| **是否需要策略**   | 需要使用策略概率，如 $\pi(a \mid s)$            | 有些方法（如 Q-learning）在 off-policy 下不显式使用策略概率 |


---

## 总结：

* 贝尔曼方程是强化学习的核心，可以写为基于 $V^\pi(s)$ 或 $Q^\pi(s,a)$ 的形式。
* 这两种形式本质相同，视任务目标而选：

  * 想评估状态值 ⇒ 用 $V^\pi(s)$
  * 想比较动作优劣、做决策 ⇒ 用 $Q^\pi(s,a)$


$V^\pi(s)$用于评估策略的好坏，$Q^\pi(s,a)$用于寻找最优策略。

